# Задание

Реализовать классический метод ветвей и границ, в котором нужно ветвиться по дробным переменным.

# Настройки/Гиперпараметры/Импорты

In [2]:
import numpy as np # для быстрой работы с массивами
import random # для рандомизированного алгоритма
import time # для подсчёта времени работы
from tqdm import tqdm # для обёртки итераций между датасетами
import csv # для сохранения ответов
import pandas as pd # для вывода таблицы (v 2.0.0)

from docplex.mp.model import Model # импортирование cplex модели-функции

In [3]:
runs = 5 # число запусков для усреднения времени
iterations = 1000 # сколько попыток делать для поиска хорошего начального решения

## Вспомогательные функции

In [4]:
def check_solution(edges: dict, solution) -> None:
    """
    Функция для проверка полученного лучшего решения.\n
    Parameters:
        * edges: словарь смежных вершинам вершин, описывающий граф
        * solution: решение в формате (размер клики, [вершины в клике])\n
    Returns:
        * None: выкинет ошибку, если решение не действительное
    """
    for i in range(solution[0]-1): # идём по вершинам и проверяем, смежны ли она со всеми остальными вершинами в найденной клике (solution[0] - число вершин в найденной клике)
        for j in range(i+1, solution[0]): # идём по последующим вершинам в клике
            if solution[1][j] not in edges[solution[1][i]]: # проверяем наличие ребра между вершинами
                raise RuntimeError("Clique contains unconnected vertices!") # если ребра нет — выкидываем ошибку
            

def transform_solution(solution) -> list:
    """
    Функция для преобразования ответа, чтобы номера шли не с 0, а с 1 и по порядку.\n
    Parameters:
        * solution: решение в формате (размер клики, [вершины в клике])\n
    Returns:
        * list: решение в формате (размер клики, [отсортированные инкрементированные вершины клики])
    """
    for i in range(solution[0]):
        solution[1][i] += 1
    return [solution[0], sorted(solution[1])]


def save_solution(solutions):
    """
    Функция для сохранения лучших ответов.\n
    Parameters:
        * solutions: словарь для тест-кейсов с решениями в формате (размер клики, [вершины в клике])\n
    Returns:
        * None: сохраняет файл
    """
    for dataset in solutions.keys(): # идём по тест-кейсам
        with open(f'solutions/{dataset}.csv', 'w', newline='') as file: # открываем файл для чистой записи
            writer = csv.writer(file) # создаём объект для записи
            writer.writerow([solutions[dataset]["clique_size"]]) # сохраняем размер клики
            writer.writerows([solutions[dataset]["clique"]]) # сохраняем вершины клики
    
    # сохранение таблицы в csv формате
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём pandas таблицу
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["clique_size"], solutions[dataset]["clique"]]], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём "новую" таблицу для тест-кейса
        table = pd.concat([table, testcase], ignore_index=True) # объединяем таблицы
    table.to_csv("best_solutions.csv", index=False) # сохраняем в csv формате без индекса

## Считывание данных

In [5]:
# files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "keller4", "MANN_a9", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "p_hat300-3", "san200_0.7_1", "san200_0.7_2", "san200_0.9_1", "san200_0.9_2", "san200_0.9_3", "sanr200_0.7"] # файлы, на которых должен быть протестирован код
# files = ["brock200_2", "brock200_3", "brock200_4", "johnson8-2-4", "johnson16-2-4", "MANN_a9", "hamming8-4", "keller4"] # файлы, на которых должен быть протестирован код
files = ["brock200_2", "brock200_3", "johnson8-2-4", "johnson16-2-4", "MANN_a9", "keller4"] # файлы, на которых должен быть протестирован код

In [6]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#     "edge_num": число рёбер, 
#     "edges": 
#         {словарь вида вершина - set смежных ей вершин}
#     }
#  ...
# }

In [7]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open("data/" + file + ".clq", "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1 # запоминаем вершины (-1, чтобы не было мороки с индексацией)

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё set с указанием v2
                    data[file]["edges"][v1] = {v2}
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в set смежных вершин v1
                    data[file]["edges"][v1].add(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = {v1}
                elif v1 not in data[file]["edges"][v2]:
                    data[file]["edges"][v2].add(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины в словаре (в set для ключа словаря вершины уже отсортированы)

In [8]:
data["johnson8-2-4"] # пример данных

{'vertex_num': 28,
 'edge_num': 210,
 'edges': {0: {5, 8, 9, 12, 13, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27},
  1: {4, 7, 9, 11, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 27},
  2: {3, 6, 9, 10, 13, 14, 15, 18, 19, 20, 21, 24, 25, 26, 27},
  3: {2, 7, 8, 11, 12, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27},
  4: {1, 6, 8, 10, 12, 14, 15, 17, 19, 20, 21, 23, 25, 26, 27},
  5: {0, 6, 7, 10, 11, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27},
  6: {2, 4, 5, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27},
  7: {1, 3, 5, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 26, 27},
  8: {0, 3, 4, 10, 11, 13, 15, 16, 18, 20, 21, 22, 24, 26, 27},
  9: {0, 1, 2, 10, 11, 12, 15, 16, 17, 20, 21, 22, 23, 26, 27},
  10: {2, 4, 5, 7, 8, 9, 16, 17, 18, 19, 22, 23, 24, 25, 27},
  11: {1, 3, 5, 6, 8, 9, 15, 17, 18, 19, 21, 23, 24, 25, 27},
  12: {0, 3, 4, 6, 7, 9, 15, 16, 18, 19, 21, 22, 24, 25, 27},
  13: {0, 1, 2, 6, 7, 8, 15, 16, 17, 19, 21, 22, 23, 25, 27},
  14: {0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 21, 22, 23, 24, 27},
  15

# Реализация алгоритма

## Эвристики для поиска начального решения

In [9]:
def randomized_greedy_max_clique(edges:dict, iterations=10) -> tuple:
    """
    Функция для получения начального рандомизированного решения задачи о максимальной клике.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * iterations: через сколько попыток без улучшения решения выходить из алгоритма\n
    Returns:
        * tuple: (размер лучшей найденной клики, список вершин в этой клике)
    """
    original_candidates = set(edges.keys()) # set вершин (изначально все являются кандидатами в клику)
    original_candidates_degrees = [len(edges[v]) for v in original_candidates] # создаём список степеней вершин (индекс - номер вершины, так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)

    attempts = 0 # текущее число попыток
    best_clique = [] # текущая лучшая клика

    while attempts < iterations: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик iterations
        clique = [] # создаём "пустую" клику
        candidates = original_candidates.copy() # копируем всех кандидатов
        while len(candidates) != 0: # пока есть кандидаты — пытаемся добавить их в клику
            candidates_degrees = [original_candidates_degrees[i] for i in candidates] # пересчитываем степени кандидатов (оставляем степени только рассматриваемых вершин) для итерациии случайного выбора
            
            v = random.choices(population=list(candidates), weights=candidates_degrees, k=1)[0] # случайным образом выбираем вершину в клику в соответствии с её степенью (чем больше степень относительно других вершин — тем выше вероятность) (переводим candidates в список для случайного выбора)
            clique.append(v) # добавляем её в клику
            
            candidates = candidates.intersection(edges[v]) # среди кандидитов оставляем только тех, кто смежен со всеми вершинами в текущей клике (итеративно этот список постоянно уменьшается с добавлением новых вершин в клику)

        if len(clique) > len(best_clique): # если нашли новую лучшую клику, то запоминаем её
            best_clique = clique.copy()
            attempts = 0 # обнуляем число итераций без улучшения решения
        else:
            attempts += 1 # увеличиваем число итераций без улучшения решения

    return len(best_clique), best_clique # возвращаем размер лучшей клики и её саму

## Branch and Bound алгоритм

### 1) Branch and Bound для задачи о максимальной клике (без использования cplex)

In [14]:
class BranchAndBound_clq():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения\n
        Returns:
            * None: создаёт модель
        """
        # вершины в алгоритм приходят с нулевой (а не с 1)
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе
        self.best_clique = [] # текущая лучшая клика
        self.cur_clique = [] # текущая рассматриваемая клика

    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет best_clique
        """
        _, self.best_clique = self.heuristic_for_init_sol(self.edges, *data) # запоминаем содержимое найденной клики (размер будем брать через len, так как это всё равно за O(1))

    
    def initialize_bnb(self) -> None:
        """
        Функция инициализации Branch and Bound алгоритма для точного поиска максимальной клики.\n
        Returns:
            * None: обновляет best_clique
        """
        #===================================== случайным образом перемешиваем вершины-кандидатов==========================================
        candidates = list(range(self.num_vertices)) # изначально кандидаты в клику — все возможные вершины
        random.shuffle(candidates) # случайно перемешиваем кандидатов
        self.recursive_bnb(candidates) # запускаем рекурсивный BnB алгоритм (DFS search)
        #------------------------------------- сортируем smallest degree last with remove + reverse --------------------------------------
        edges_sorted = {}
        vertices_degrees = [len(self.edges[v]) for v in range(self.num_vertices)] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
        for i in range(self.num_vertices): # просто итерируемся столько раз, сколько у нас вершин
            vertex = vertices_degrees.index(max(vertices_degrees)) # берём вершину с самой большой степенью
            vertices_degrees[vertex] = -1 # ставим малую степень найденной вершине (она больше ни разу не будет выбрана)
            edges_sorted[vertex] = self.edges[vertex+1] # копируем рассматриваемую вершину в новый-отсортированный словарь
            for v in edges[vertex+1]: # идём по смежным вершинам
                vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)

    
    def recursive_bnb(self, candidates) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * candidates: возможные кандидаты для добавления в клику\n
        Returns:
            * None: обновляет best_clique
        """

        if len(candidates) == 0: # если кандидатов не осталось — мы в листе BnB алгоритма
            if len(self.cur_clique) > len(self.best_clique): # если клика в листе получилась лучше — обновляем лучшую
                self.best_clique = self.cur_clique.copy() # запоминаем содержимое клики
            return # выходим из рекурсии, так как кандидатов не осталось
        
        if len(self.cur_clique) + len(candidates) <= len(self.best_clique): # проверка на UB (перестаём рассматривать ветку, если её ответ в любом случае не улучшит максимальную клику)
            return
        
        for i in range(len(candidates)): # идём по возможным кандидатам (ветвление)
            new_candidates = []
            for j in range(i+1, len(candidates)): # идём по оставшимся кандидатам
                # оставляем в новых кандидатах только те вершины, у которых есть ребро с добавляемой вершиной "i" (с вершинами в текущей клике у кандидатов уже проверено наличие ребра)
                if candidates[j] in self.edges[candidates[i]]: # проверяем, есть ли ребро между добавляемым кандидатом i и его "кандидатом по соседству" j
                    new_candidates.append(candidates[j]) # если ребро есть, значит эту вершину j мы можем добавить на следующей итерации алгоритма (j останется в кандидатах на следующем уровне DFS)
            self.cur_clique.append(candidates[i]) # добавляем вершину i в клику
            self.recursive_bnb(new_candidates) # рекурсивно вызываем BnB для оставшихся кандидатов (идём вглубь)
            self.cur_clique.remove(candidates[i]) # убираем добавленную вершину в клику для перехода к следующему (соседнему) элементу в DFS рекурсии
    

    def get_best_solution(self) -> tuple:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Returns:
            * tuple: решение вида (размер клики, [вершины в клике])
        """
        return len(self.best_clique), self.best_clique

### 2) Branch and Bound общего назначения (с использованием cplex)

In [ ]:
class BranchAndBound():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения\n
        Returns:
            * None: создаёт модель
        """
        self.model = Model(name="Linear Program") # cplex модель для задачи
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе
        self.best_clique = [] # текущая лучшая клика
        self.cur_clique = [] # текущая рассматриваемая клика

        self.vars = np.full(shape=(self.num_vertices), fill_value=None) # создаём массив под переменные-вершины
        for v in range(self.num_vertices): # добавляем в модель непрерывные переменные
            self.vars[v] = self.model.continuous_var(name=f"x{v}", lb=0, ub=1) # добавляем непрерывные переменные-вершины x{v}, v от 0 до число_вершин_в_графе-1 (-1 из-за нумерации с нуля), что могут принимать значение в диапазоне [lb=0, ub=1]

        self.model.set_objective("max", sum(self.vars)) # добавляем целевую функцию в cplex модель - максимизация суммы переменных


    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет best_clique
        """
        _, self.best_clique = self.heuristic_for_init_sol(self.edges, *data) # запоминаем содержимое найденной клики (размер будем брать через len, так как это всё равно за O(1))

    
    def initialize_bnb(self) -> None:
        """
        Функция инициализации Branch and Bound алгоритма для точного поиска максимальной клики.\n
        Returns:
            * None: обновляет best_clique
        """
        candidates = list(range(self.num_vertices)) # изначально кандидаты в клику — все возможные вершины
        random.shuffle(candidates) # случайно перемешиваем кандидатов
        self.recursive_bnb(candidates) # запускаем рекурсивный BnB алгоритм (DFS search)

    
    def recursive_bnb(self, candidates) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * candidates: возможные кандидаты для добавления в клику\n
        Returns:
            * None: обновляет best_clique
        """

        if len(candidates) == 0: # если кандидатов не осталось — мы в листе BnB алгоритма
            if len(self.cur_clique) > len(self.best_clique): # если клика в листе получилась лучше — обновляем лучшую
                self.best_clique = self.cur_clique.copy() # запоминаем содержимое клики
            return # выходим из рекурсии, так как кандидатов не осталось
        
        if len(self.cur_clique) + len(candidates) <= len(self.best_clique): # проверка на UB (перестаём рассматривать ветку, если её ответ в любом случае не улучшит максимальную клику)
            return
        
        for i in range(len(candidates)): # идём по возможным кандидатам (ветвление)
            new_candidates = []
            for j in range(i+1, len(candidates)): # идём по оставшимся кандидатам
                # оставляем в новых кандидатах только те вершины, у которых есть ребро с добавляемой вершиной "i" (с вершинами в текущей клике у кандидатов уже проверено наличие ребра)
                if candidates[j] in self.edges[candidates[i]]: # проверяем, есть ли ребро между добавляемым кандидатом i и его "кандидатом по соседству" j
                    new_candidates.append(candidates[j]) # если ребро есть, значит эту вершину j мы можем добавить на следующей итерации алгоритма (j останется в кандидатах на следующем уровне DFS)
            self.cur_clique.append(candidates[i]) # добавляем вершину i в клику
            self.recursive_bnb(new_candidates) # рекурсивно вызываем BnB для оставшихся кандидатов (идём вглубь)
            self.cur_clique.remove(candidates[i]) # убираем добавленную вершину в клику для перехода к следующему (соседнему) элементу в DFS рекурсии
    

    def get_best_solution(self) -> tuple:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Returns:
            * tuple: решение вида (размер клики, [вершины в клике])
        """
        return len(self.best_clique), self.best_clique

# Тестирование

In [20]:
solutions = {} 
# словарь для ответов алгоритма
# {"название датасета" : 
#     {"clique_size": размер клики,
#      "clique": [вершины, входящие в клику],
#      "time": время на подсчёт
#     }
# }

In [21]:
runs = 1

In [24]:
for dataset in tqdm(data.keys()): # идём по тест-кейсам
# for dataset in ["johnson8-2-4", "brock200_2", "brock200_3"]:
    # print(dataset)
    time_start = time.time() # замеряем время начала выполнения
    for i in range(runs): # делаем runs запусков для усреднения времени
        bnb_model = BranchAndBound(edges=data[dataset]["edges"], heuristic_for_init_sol=randomized_greedy_max_clique) # создаём объект для модели
        bnb_model.calc_initial_solution([iterations]) # запускаем поиск начального решения с передачей следующих параметров - (iterations)
        # print(f"эвристическое решение {bnb_model.get_best_solution()}")
        bnb_model.initialize_bnb() # запускаем Branch and Bound алгоритм
        # print(f"BnB решение {bnb_model.get_best_solution()}")
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    solution = bnb_model.get_best_solution() # берём полученное решение у модели
    check_solution(edges=data[dataset]["edges"], solution=solution) # проверка решения
    # # print("original solution", sol)
    solution = transform_solution(solution) # сортирует вершины клики в порядке возрастания их номера и возвращает нумерацию с единицы
    # print("transformed solution", sol)
    solutions[dataset] = {"clique_size": solution[0], "clique": solution[1], "time": time_end/runs} # добавление

  0%|          | 0/6 [00:00<?, ?it/s]

brock200_2
эвристическое решение (12, [104, 182, 134, 54, 120, 119, 26, 148, 47, 144, 157, 69])


 17%|█▋        | 1/6 [00:04<00:21,  4.28s/it]

BnB решение (12, [104, 182, 134, 54, 120, 119, 26, 148, 47, 144, 157, 69])
brock200_3
эвристическое решение (13, [50, 188, 59, 165, 146, 38, 82, 100, 132, 152, 47, 56, 28])


 33%|███▎      | 2/6 [01:07<02:35, 38.87s/it]

...

BnB решение (8, [68, 109, 10, 16, 44, 98, 88, 58])
MANN_a9
эвристическое решение (16, [4, 8, 12, 36, 32, 39, 42, 1, 5, 24, 35, 21, 9, 28, 16, 18])


 83%|████████▎ | 5/6 [02:45<00:32, 32.88s/it]

BnB решение (16, [4, 8, 12, 36, 32, 39, 42, 1, 5, 24, 35, 21, 9, 28, 16, 18])
keller4
эвристическое решение (11, [95, 65, 66, 139, 149, 167, 155, 134, 46, 48, 41])


100%|██████████| 6/6 [03:46<00:00, 37.72s/it]

BnB решение (11, [95, 65, 66, 139, 149, 167, 155, 134, 46, 48, 41])


In [ ]:
solutions

{'brock200_2': {'clique_size': 10,
  'clique': [61, 84, 90, 102, 108, 112, 132, 161, 183, 194],
  'time': 2.9272008419036863},
 'brock200_3': {'clique_size': 13,
  'clique': [34, 42, 80, 81, 85, 97, 130, 133, 136, 147, 165, 173, 197],
  'time': 31.692399168014525},
 'johnson8-2-4': {'clique_size': 4,
  'clique': [2, 8, 21, 25],
  'time': 0.04351387023925781},
 'johnson16-2-4': {'clique_size': 8,
  'clique': [6, 20, 23, 34, 66, 76, 105, 106],
  'time': 28.633553171157835},
 'MANN_a9': {'clique_size': 16,
  'clique': [2, 4, 5, 8, 10, 15, 18, 21, 22, 26, 30, 33, 35, 38, 40, 45],
  'time': 1.4123371124267579},
 'keller4': {'clique_size': 11,
  'clique': [33, 35, 38, 47, 52, 54, 143, 144, 149, 165, 168],
  'time': 25.853962850570678}}

In [55]:
bnb_model.model.solve()
bnb_model.model.print_solution()

objective: 28.000
  x0=1.000
  x1=1.000
  x2=1.000
  x3=1.000
  x4=1.000
  x5=1.000
  x6=1.000
  x7=1.000
  x8=1.000
  x9=1.000
  x10=1.000
  x11=1.000
  x12=1.000
  x13=1.000
  x14=1.000
  x15=1.000
  x16=1.000
  x17=1.000
  x18=1.000
  x19=1.000
  x20=1.000
  x21=1.000
  x22=1.000
  x23=1.000
  x24=1.000
  x25=1.000
  x26=1.000
  x27=1.000


In [30]:
def smallest_degree_last_with_remove_v1(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    vertices_degrees_ids = sorted(range(len(vertices_degrees)), key=vertices_degrees.__getitem__, reverse=True) # создаём список номеров вершин (-1) в порядке уменьшения их степеней
    max_id = len(vertices_degrees_ids) - 1 # максимальное id, до которого можно двигаться вправо в списке

#    print("edges", edges)
#    print("vertices_degrees:    ", vertices_degrees)
#    print("vertices_degrees_ids:", vertices_degrees_ids)

    for id in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees_ids[id] # рассматриваемая вершина (её номер на 1 меньше, чем должен быть из-за нумерации с нуля) с максимальной степенью
        edges_sorted[vertex+1] = edges[vertex+1] # копируем её в новый-отсортированный словарь

        # как бы убираем рассматриваемую вершину из графа
        vertices_degrees[vertex] = -1 # ставим этой вершине степень -1 (для упрощения дальнейшей работы, так как у всех нерассмотренных вершин степень не будет меньше нуля)
        for v in edges[vertex+1]: # идём по смежным вершинам
#            print("before change")
#            print("vertex", vertex, "Рассматриваемая вершина", vertex+1)
#            print("v                 смежная вершина", v)
#            print("vertices_degrees     ", vertices_degrees)
#            print("vertices_degrees_ids:", vertices_degrees_ids)

            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)

            # а также меняем их позицию в списке vertices_degrees_ids
            v_in_vertices_degrees_ids = vertices_degrees_ids.index(v-1) # находим позицию смежной вершины в списке вершин, отсортированных по убыванию их степеней (чтобы потом относительно неё рассматривать вершины с меньшей степенью)
            
#            print("v_in_vertices_degrees_ids", v_in_vertices_degrees_ids)

            while (v_in_vertices_degrees_ids < max_id) and (-1 < vertices_degrees[v-1] < vertices_degrees[vertices_degrees_ids[v_in_vertices_degrees_ids+1]]): # пока у вершины правее степень больше, сдвигаем рассматриваемую смежную в сторону уменьшения степени

                vertices_degrees_ids[v_in_vertices_degrees_ids], vertices_degrees_ids[v_in_vertices_degrees_ids+1] = vertices_degrees_ids[v_in_vertices_degrees_ids+1], vertices_degrees_ids[v_in_vertices_degrees_ids] # делаем swap вершин в списке, отсортированном по убыванию их степеней
                v_in_vertices_degrees_ids += 1 # переходим к вершине правее (чья степень меньше или равна рассматриваемой смежной)
#            print("after change")
#            print("vertices_degrees     ", vertices_degrees)
#            print("vertices_degrees_ids:", vertices_degrees_ids)
#        print("-------------------------")
#    print("vertices_degrees", vertices_degrees)
#    print("edges_sorted", edges_sorted)
    return edges_sorted


def smallest_degree_last_with_remove_v2(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees.index(max(vertices_degrees)) # берём вершину с самой большой степенью
        vertices_degrees[vertex] = -1 # ставим малую степень найденной вершине (она больше ни разу не будет выбрана)
        edges_sorted[vertex+1] = edges[vertex+1] # копируем рассматриваемую вершину в новый-отсортированный словарь
        for v in edges[vertex+1]: # идём по смежным вершинам
            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)
#    print("edges_sorted", edges_sorted)
    return edges_sorted

In [29]:
edges = {1:{2,3,4,6}, 2:{1,3,4}, 3:{1,2,4}, 4:{1,2,3,5}, 5:{4,7,8,9,10,11}, 6:{1,12,13,14,15,16}, 7:{5}, 8:{5}, 9:{5}, 10:{5}, 11:{5}, 12:{6}, 13:{6}, 14:{6}, 15:{6}, 16:{6}}
smallest_degree_last_with_remove_v1(edges)

{5: {4, 7, 8, 9, 10, 11},
 6: {1, 12, 13, 14, 15, 16},
 1: {2, 3, 4, 6},
 4: {1, 2, 3, 5},
 3: {1, 2, 4},
 2: {1, 3, 4},
 16: {6},
 15: {6},
 14: {6},
 13: {6},
 12: {6},
 11: {5},
 10: {5},
 9: {5},
 8: {5},
 7: {5}}

In [31]:
edges = {1:{2,3,4,6}, 2:{1,3,4}, 3:{1,2,4}, 4:{1,2,3,5}, 5:{4,7,8,9,10,11}, 6:{1,12,13,14,15,16}, 7:{5}, 8:{5}, 9:{5}, 10:{5}, 11:{5}, 12:{6}, 13:{6}, 14:{6}, 15:{6}, 16:{6}}
smallest_degree_last_with_remove_v2(edges)

{5: {4, 7, 8, 9, 10, 11},
 6: {1, 12, 13, 14, 15, 16},
 1: {2, 3, 4, 6},
 2: {1, 3, 4},
 3: {1, 2, 4},
 4: {1, 2, 3, 5},
 7: {5},
 8: {5},
 9: {5},
 10: {5},
 11: {5},
 12: {6},
 13: {6},
 14: {6},
 15: {6},
 16: {6}}

In [ ]:
def smallest_degree_last_with_remove_v2(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees.index(max(vertices_degrees)) # берём вершину с самой большой степенью
        vertices_degrees[vertex] = -1 # ставим малую степень найденной вершине (она больше ни разу не будет выбрана)
        edges_sorted[vertex+1] = edges[vertex+1] # копируем рассматриваемую вершину в новый-отсортированный словарь
        for v in edges[vertex+1]: # идём по смежным вершинам
            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)
#    print("edges_sorted", edges_sorted)
    return edges_sorted

In [50]:
def smallest_degree_last_with_remove_v3(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин
    vertices_sorted = np.full(shape=(len(vertices)), fill_value=-1, dtype=np.int16) # заготовка под номера вершин (массив пока заполнен значением -1)

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)-1, -1, -1): # просто итерируемся столько раз, сколько у нас вершин (но с конца: len(vertices)-1 - максимальный индекс, с которого будем идти (-1 чтобы не выйти за рамки массива); -1 по какой индекс идём не включительно (то есть до нуля); -1 шаг)
        vertex = vertices_degrees.index(min(vertices_degrees)) # берём вершину с самой малой степенью
        vertices_sorted[i] = vertex # записываем вершину на i-ую позицию с конца
        vertices_degrees[vertex] = np.inf # ставим такую степень найденной вершине, чтобы она больше ни разу не была выбрана
        for v in edges[vertex]: # идём по смежным вершинам
            vertices_degrees[v] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)

    for vertex in vertices_sorted: # идём по отсортированному smallest degree last with remove массиву вершин
        edges_sorted[vertex+1] = edges[vertex+1] # копируем вершину в новый-отсортированный словарь

#    print("edges_sorted", edges_sorted)
    return edges_sorted

In [51]:
edges = {0:{1,2,3,5}, 1:{0,2,3}, 2:{0,1,3}, 3:{0,1,2,4}, 4:{3,6,7,8,9,10}, 5:{0,11,12,13,14,15}, 6:{4}, 7:{4}, 8:{4}, 9:{4}, 10:{4}, 11:{5}, 12:{5}, 13:{5}, 14:{5}, 15:{5}}
smallest_degree_last_with_remove_v3(edges)

{3: {0, 1, 2, 4},
 2: {0, 1, 3},
 1: {0, 2, 3},
 0: {1, 2, 3, 5},
 5: {0, 11, 12, 13, 14, 15},
 15: {5},
 14: {5},
 13: {5},
 12: {5},
 11: {5},
 4: {3, 6, 7, 8, 9, 10},
 10: {4},
 9: {4},
 8: {4},
 7: {4},
 6: {4}}

In [52]:
edges = {1:{2,3,4,6}, 2:{1,3,4}, 3:{1,2,4}, 4:{1,2,3,5}, 5:{4,7,8,9,10,11}, 6:{1,12,13,14,15,16}, 7:{5}, 8:{5}, 9:{5}, 10:{5}, 11:{5}, 12:{6}, 13:{6}, 14:{6}, 15:{6}, 16:{6}}
smallest_degree_last_with_remove_v3(edges)

IndexError: list index out of range